In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import re,string,unicodedata
from keras.preprocessing import text, sequence
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split
from string import punctuation
from nltk import pos_tag
from nltk.corpus import wordnet
import keras
from keras.models import Sequential
from keras.layers import Dense,Embedding,LSTM,Dropout
from keras.callbacks import ReduceLROnPlateau
import tensorflow as tf

In [2]:
true = pd.read_csv("True.csv")
false = pd.read_csv("Fake.csv")

In [3]:
true['category'] = 1
false['category'] = 0

In [4]:
df = pd.concat([true,false]) #Merging the 2 datasets
print(df)

                                                   title  \
0      As U.S. budget fight looms, Republicans flip t...   
1      U.S. military to accept transgender recruits o...   
2      Senior U.S. Republican senator: 'Let Mr. Muell...   
3      FBI Russia probe helped by Australian diplomat...   
4      Trump wants Postal Service to charge 'much mor...   
...                                                  ...   
23476  McPain: John McCain Furious That Iran Treated ...   
23477  JUSTICE? Yahoo Settles E-mail Privacy Class-ac...   
23478  Sunnistan: US and Allied ‘Safe Zone’ Plan to T...   
23479  How to Blow $700 Million: Al Jazeera America F...   
23480  10 U.S. Navy Sailors Held by Iranian Military ...   

                                                    text       subject  \
0      WASHINGTON (Reuters) - The head of a conservat...  politicsNews   
1      WASHINGTON (Reuters) - Transgender people will...  politicsNews   
2      WASHINGTON (Reuters) - The special counsel inv... 

In [5]:
df['text'] = df['title']
del df['title']
del df['subject']
del df['date']

In [6]:
stop = set(stopwords.words('english'))
punctuation = list(string.punctuation)
stop.update(punctuation)

In [7]:
pd.options.display.max_colwidth
pd.set_option('display.max_colwidth', None)
# print(df['text'][0])

In [8]:
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
#     print(soup)
    return soup.get_text()
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)
def remove_between_square_brackets(text):
    return re.sub(r'http\S+', '', text)
def remove_stopwords(text):
    final_text = []
    for i in text.split():
#         print(i)
        if i.strip().lower() not in stop:
#             print(i.strip().lower())
            final_text.append(i.strip())
    return " ".join(final_text)
def remove_input_stopwords(text):
    final_text = []
    for i in text.split(" "):
#         print(i)
        if i.strip().lower() not in stop:
#             print(i.strip().lower())
            final_text.append(i.strip())
    return " ".join(final_text)
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    text = remove_stopwords(text)
    return text
df['text']=df['text'].apply(denoise_text)

C:\Users\Adrian.LAPTOP-ISI0LQ6K\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\bs4\__init__.py:417: MarkupResemblesLocatorWarning: "https://100percentfedup.com/served-roy-moore-vietnamletter-veteran-sets-record-straight-honorable-decent-respectable-patriotic-commander-soldier/" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  warnings.warn(
C:\Users\Adrian.LAPTOP-ISI0LQ6K\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\bs4\__init__.py:417: MarkupResemblesLocatorWarning: "https://100percentfedup.com/video-hillary-asked-about-trump-i-just-want-to-eat-some-pie/" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, 

In [9]:
# print(df['text'][0])

In [13]:
x_train,x_test,y_train,y_test = train_test_split(df.text,df.category,random_state = 0)

In [14]:
max_features = 10000
maxlen = 300

In [46]:
print(type(x_train))

<class 'numpy.ndarray'>


In [45]:
tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(x_train)
tokenized_train = tokenizer.texts_to_sequences(x_train)
x_train = sequence.pad_sequences(tokenized_train, maxlen=maxlen)

AttributeError: 'numpy.ndarray' object has no attribute 'lower'

In [43]:
print((x_train[0]))

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 

In [18]:
# np.set_printoptions(threshold=np.inf)
# print((x_train[100]))

In [64]:
tokenized_test = tokenizer.texts_to_sequences(x_test)
X_test = sequence.pad_sequences(tokenized_test, maxlen=maxlen)

In [65]:
print(tokenized_test)
print(X_test[0])

[[], [], [], [], [6, 10], [6, 10], [], [5], [], [], [], [6, 10], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [6, 10], [6, 3], [], [], [], [], [], [], [], [], [], [], [], [], [], [6, 10], [], [], [], [], [], [], [], [], [], [], [], [6, 10], [], [], [], [], [6, 3], [], [], [], [], [], [], [], [], [], [], [5], [6, 10], [6, 10], [15], [], [1, 10], [], [], [], [], [], [], [], [], [], [6, 10], [], [6, 10, 1], [], [], [6, 3], [], [6, 10], [], [], [], [], [], [], [], [], [], [], [], [6, 10], [], [6, 3], [], [], [], [], [], [], [], [], [], [], [], [], [], [6, 10], [], [], [6, 10], [], [], [], [], [], [6, 10], [6, 10], [], [], [], [6, 10], [], [6, 10], [], [], [6, 3], [6, 10], [], [], [6, 10], [], [], [], [], [], [6, 10], [], [], [], [6, 10], [], [], [], [], [], [], [], [6, 10], [], [], [], [], [], [6, 3], [], [], [], [], [13, 9], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [6, 10], [], [], [6, 10

In [20]:
EMBEDDING_FILE = 'glove.twitter.27B.100d.txt'

In [21]:
def get_coefs(word, *arr): 
    return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.rstrip().rsplit(' ')) for o in open(EMBEDDING_FILE, encoding='utf8'))

In [22]:
all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
embed_size = all_embs.shape[1]

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

C:\Users\Adrian.LAPTOP-ISI0LQ6K\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\IPython\core\interactiveshell.py:3357: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  if (await self.run_code(code, result,  async_=asy)):


In [23]:
batch_size = 256
epochs = 10
embed_size = 100

In [24]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', patience = 2, verbose=1,factor=0.5, min_lr=0.00001)

In [25]:
model = Sequential()
model.add(Embedding(max_features, output_dim=embed_size, weights=[embedding_matrix], input_length=maxlen, trainable=False))
model.add(LSTM(units=128 , return_sequences = True , recurrent_dropout = 0.25 , dropout = 0.25))
model.add(LSTM(units=64 , recurrent_dropout = 0.1 , dropout = 0.1))
model.add(Dense(units = 32 , activation = 'relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer=keras.optimizers.Adam(lr = 0.01), loss='binary_crossentropy', metrics=['accuracy'])

In [26]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 300, 100)          1000000   
_________________________________________________________________
lstm (LSTM)                  (None, 300, 128)          117248    
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense (Dense)                (None, 32)                2080      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 1,168,769
Trainable params: 168,769
Non-trainable params: 1,000,000
_________________________________________________________________


In [27]:
history = model.fit(x_train, y_train, batch_size = batch_size , validation_data = (X_test,y_test) , epochs = epochs , callbacks = [learning_rate_reduction])

Epoch 1/10
132/132 [==============================] - 354s 2s/step - loss: 0.3672 - accuracy: 0.8096 - val_loss: 0.1918 - val_accuracy: 0.9207
Epoch 2/10
132/132 [==============================] - 330s 3s/step - loss: 0.1585 - accuracy: 0.9382 - val_loss: 0.1380 - val_accuracy: 0.9458
Epoch 3/10
132/132 [==============================] - 340s 3s/step - loss: 0.1266 - accuracy: 0.9502 - val_loss: 0.1227 - val_accuracy: 0.9539
Epoch 4/10
132/132 [==============================] - 429s 3s/step - loss: 0.1081 - accuracy: 0.9588 - val_loss: 0.1189 - val_accuracy: 0.9549
Epoch 5/10
132/132 [==============================] - 440s 3s/step - loss: 0.0903 - accuracy: 0.9647 - val_loss: 0.1156 - val_accuracy: 0.9563
Epoch 6/10
132/132 [==============================] - 457s 3s/step - loss: 0.0819 - accuracy: 0.9694 - val_loss: 0.1129 - val_accuracy: 0.9575
Epoch 7/10
132/132 [==============================] - 440s 3s/step - loss: 0.0745 - accuracy: 0.9695 - val_loss: 0.1191 - val_accuracy: 0.9572

In [56]:
pred = model.predict_classes(X_test)
pred[:5]

array([[1],
       [0],
       [0],
       [0],
       [1]])

In [66]:
user_input = ["Can the US avoid another Trump"]
tokenized_test = tokenizer.texts_to_sequences(user_input)
print(tokenized_test)
user_input_tokenized = sequence.pad_sequences(tokenized_test, maxlen=maxlen)
input_pred = model.predict_classes(user_input_tokenized)
print(input_pred)

[[]]
[[0]]


In [30]:
print(tokenizer)

In [31]:
pretrained_model = tf.keras.applications.MobileNet()
tf.saved_model.save(pretrained_model, 'saved_model')

INFO:tensorflow:Assets written to: saved_model\assets


In [32]:
converter = tf.lite.TFLiteConverter.from_saved_model("saved_model")
tflite_model = converter.convert()

In [33]:
import pathlib
tflite_model_file = pathlib.Path("fakenews.tflite")
tflite_model_file.write_bytes(tflite_model)

16896516